# Leitura das bibliotecas e configurações

In [1]:
#Usar google drive no google colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import operator
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

path = "/content/drive/MyDrive/db_tcc"

# Preparaçao bases

In [3]:
df_metrics = pd.read_csv(f"{path}/all_metrics_filter.csv", sep=";")#usuarios que tem informaçao de videos

df_metrics = df_metrics [['broadcaster_id', 'total_followers', 'team_id', 'qtd_emotes', 'Português', 'Brasil',
       'Portugal', 'Competitivo', '100%', 'AMA', 'Multijogador', 'LGBTQIA+',
       'Jogada casual', 'Drops habilitados', 'Conversação', 'PvP',
       'Jogando com espectadores', 'Anime', 'Jogada de ranking',
       'Atuação e encenação', 'PC', 'Jogadores iniciantes', 'Primeira jogada',
       'Futebol', 'Aconchegante', 'Relaxada', 'Inglês', 'Único jogador',
       'Modo: Battle Royale', 'Repetir', 'Vtuber', 'Cooperativo',
       'Jogador X ambiente', 'Modo: Aventura', 'Depressão', 'Esports',
       'Modo: Sobrevivência', 'Retrô', 'Jogo de guerra', 'Mapa: Mirage',
       'Arte digital', 'Mulher', 'Desenho', 'ASMR auditivo',
       'Para toda a família', 'Corrida', 'EUA', 'TDAH', 'Extensão aprimorada',
       'Modo difícil', 'Inglaterra', 'Animada', 'Sem palpites',
       'Espaço seguro', 'menor_trecho_game', 'maior_trecho_game',
       'game_duration_sum_rank_1', 'game_duration_sum_rank_2',
       'game_duration_sum_rank_3', 'game_duration_sum_rank_4',
       'game_duration_sum_rank_5', 'game_duration_sum_rank_6',
       'game_duration_sum_rank_7', 'game_duration_sum_rank_8',
       'game_duration_sum_rank_9', 'game_duration_sum_rank_10',
       'game_duration_count_rank_1', 'game_duration_count_rank_2',
       'game_duration_count_rank_3', 'game_duration_count_rank_4',
       'game_duration_count_rank_5', 'game_duration_count_rank_6',
       'game_duration_count_rank_7', 'game_duration_count_rank_8',
       'game_duration_count_rank_9', 'game_duration_count_rank_10',
       'game_duration_mean_rank_1', 'game_duration_mean_rank_2',
       'game_duration_mean_rank_3', 'game_duration_mean_rank_4',
       'game_duration_mean_rank_5', 'game_duration_mean_rank_6',
       'game_duration_mean_rank_7', 'game_duration_mean_rank_8',
       'game_duration_mean_rank_9', 'game_duration_mean_rank_10',
       'rank_jogos_recentes_1', 'rank_jogos_recentes_2',
       'rank_jogos_recentes_3', 'rank_jogos_recentes_4',
       'rank_jogos_recentes_5', 'rank_jogos_recentes_6',
       'rank_jogos_recentes_7', 'rank_jogos_recentes_8',
       'rank_jogos_recentes_9', 'rank_jogos_recentes_10']]

In [4]:
#https://stackoverflow.com/questions/63015113/am-i-clustering-users-correctly-by-using-sklearns-cosine-similarity-method-and
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

df = df_metrics.set_index('broadcaster_id')


pairwise = pd.DataFrame(
         cosine_similarity(df.iloc[:, 6:].values),
         columns = df.index.values,
         index = df.index
)

print (pairwise.round(2))

pairs = pairwise.unstack()

pairs.index.rename(['User A', 'User B'], inplace=True)
pairs = pairs.to_frame('cosine distance').reset_index()

A = pairs[(pairs['User A'] != pairs['User B'])]

A = pairs[
      (pairs['cosine distance'] < 0.00) 
      & (pairs['User A'] != pairs['User B'])
]


A

                42642296   62916113   79695458   79695943   67114748   \
broadcaster_id                                                          
42642296             1.00       0.00       0.71       0.63       0.43   
62916113             0.00       1.00       0.00       0.13       0.18   
79695458             0.71       0.00       1.00       0.89       0.61   
79695943             0.63       0.13       0.89       1.00       0.55   
67114748             0.43       0.18       0.61       0.55       1.00   
...                   ...        ...        ...        ...        ...   
130945473            0.19       0.21       0.38       0.35       0.73   
702031641            0.41       0.00       0.29       0.26       0.35   
38900392             0.93       0.00       0.76       0.68       0.53   
231313969            0.27       0.00       0.58       0.60       0.35   
222575867            0.93       0.00       0.76       0.68       0.53   

                71314574   62927223   39149175   8

,User A,User B,cosine distance
3975,62916113,72585869,-0.130258
4492,62916113,69561702,-0.129514
4698,62916113,57606120,-0.205875
4940,62916113,37285616,-0.000013
5072,62916113,41752467,-0.094291
...,...,...,...
11507208,62918711,37339794,-0.076546
11512992,673462222,47362389,-0.182574
11513001,673462222,80937493,-0.099370
11520613,452208291,45253787,-0.161607


In [5]:
def range_groups(series):
  if abs(series) >= 0 and abs(series) <= 0.1:
    return 5
  elif abs(series) > 0.1 and abs(series) <= 0.3:
    return 4
  elif abs(series) > 0.3 and abs(series) <= 0.6:
    return 3
  elif abs(series) > 0.6 and abs(series) <= 0.9:
    return 2
  elif abs(series) > 0.9 and abs(series) < 1:
    return 1
  else:
    return 0

#Quanto menor a distancia melhor o rating
A['rating'] = A['cosine distance'].apply(range_groups)
A = A.rename(columns={"User A": "user", "User B":"item"}).drop("cosine distance", axis=1)
A

<ipython-input-5-edf12a2057a9>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  A['rating'] = A['cosine distance'].apply(range_groups)


,user,item,rating
3975,62916113,72585869,4
4492,62916113,69561702,4
4698,62916113,57606120,4
4940,62916113,37285616,5
5072,62916113,41752467,5
...,...,...,...
11507208,62918711,37339794,5
11512992,673462222,47362389,4
11513001,673462222,80937493,5
11520613,452208291,45253787,4


In [ ]:
A = A.reset_index()


In [ ]:
A.to_csv(f'{path}/base_independentes/ratings_similarity.csv', mode='w', sep=";", header=True, index=False)

In [ ]:
A

,index,user,item,rating
0,1,42642296,62916113,5
1,2,42642296,79695458,2
2,3,42642296,79695943,2
3,4,42642296,67114748,3
4,5,42642296,71314574,3
...,...,...,...,...
11536207,11539603,222575867,452208291,4
11536208,11539604,222575867,130945473,3
11536209,11539605,222575867,702031641,3
11536210,11539606,222575867,38900392,0


In [12]:
A2 = pairs[
      (pairs['cosine distance'] < 0.00) 
      & (pairs['User A'] != pairs['User B'])
]

A2['rating'] = A2['cosine distance'].apply(range_groups)
A2 = A2.rename(columns={"User A": "user", "User B":"item"}).drop("cosine distance", axis=1)
A2.to_csv(f'{path}/base_independentes/ratings_similarity2.csv', mode='w', sep=";", header=True, index=False)

<ipython-input-12-b6bbd3cf5697>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  A2['rating'] = A2['cosine distance'].apply(range_groups)


In [11]:
A2

,user,item,rating
1,42642296,62916113,5
2,42642296,79695458,2
3,42642296,79695943,2
4,42642296,67114748,3
5,42642296,71314574,3
...,...,...,...
11539603,222575867,452208291,4
11539604,222575867,130945473,3
11539605,222575867,702031641,3
11539606,222575867,38900392,0


In [17]:
A3 = pairs[(pairs['User A'] != pairs['User B'])
]

A3['rating'] = A3['cosine distance'].apply(range_groups)
A3 = A3.rename(columns={"User A": "user", "User B":"item"}).drop("cosine distance", axis=1)
sample = A3.sample(n=100000, random_state=1)

sample.to_csv(f'{path}/base_independentes/ratings_sample_100k.csv', mode='w', sep=";", header=True, index=False)

<ipython-input-17-ddb7afab0bbc>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  A3['rating'] = A3['cosine distance'].apply(range_groups)


In [19]:
A3 = pairs[(pairs['User A'] != pairs['User B'])
]

A3['rating'] = A3['cosine distance'].apply(range_groups)
A3 = A3.rename(columns={"User A": "user", "User B":"item"}).drop("cosine distance", axis=1)
sample = A3.sample(n=500000, random_state=1)

sample.to_csv(f'{path}/base_independentes/ratings_sample_500k.csv', mode='w', sep=";", header=True, index=False)

<ipython-input-19-8ad749a7d121>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  A3['rating'] = A3['cosine distance'].apply(range_groups)


In [21]:
#11'536'212 
A3 = pairs[(pairs['User A'] != pairs['User B'])
]

A3['rating'] = A3['cosine distance'].apply(range_groups)
A3 = A3.rename(columns={"User A": "user", "User B":"item"}).drop("cosine distance", axis=1)
sample = A3.sample(n=1000000, random_state=1)

sample.to_csv(f'{path}/base_independentes/ratings_sample_1kk.csv', mode='w', sep=";", header=True, index=False)

<ipython-input-21-532ba93278a0>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  A3['rating'] = A3['cosine distance'].apply(range_groups)


In [22]:
#11'536'212 
A3 = pairs[(pairs['User A'] != pairs['User B'])
]

A3['rating'] = A3['cosine distance'].apply(range_groups)
A3 = A3.rename(columns={"User A": "user", "User B":"item"}).drop("cosine distance", axis=1)
sample = A3.sample(n=5000000, random_state=1)

sample.to_csv(f'{path}/base_independentes/ratings_sample_5kk.csv', mode='w', sep=";", header=True, index=False)

<ipython-input-22-e17f84b75002>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  A3['rating'] = A3['cosine distance'].apply(range_groups)
